<a href="https://colab.research.google.com/github/judith182/Estadistica-y-probabilidad/blob/main/Estad%C3%ADstica_aplicada_a_seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Solvencia II (2024)

### Estadística aplicada a seguros

### Ajuste de funciones de probabilidad

#install.packages(c("ggplot2","ggfortify","gridExtra","actuar","fitdistrplus","psych","ADGofTest","evir","evd","ismev","vcd","ChainLadder","mixtools","mclust","mixdist","eva","boot","gnFit","Pareto"))

library(actuar)
library(psych)
library(ADGofTest)
library(fitdistrplus)
library(MASS)
library(evir)
library(evd)
library(ismev)
library(vcd)
library(ChainLadder)
library(lattice)
library(mixtools)
library(mclust)
library(mixdist)
library(eva)
library(gnFit)
library(Pareto)

### Datos para ajustar alguna distribución paramética

monto<-read.csv("C:/Users/Salvador/Desktop/Solvencia2024/Montos.csv")

attach(monto)

summary(Monto)

max(Monto)-min(Monto)

sd(Monto)

hist(Monto,col="darkorange",border="white",main="Histograma: Monto",col.main="red",col.lab="blue",breaks=50)

plot(density(Monto),col="darkred",main="Densidad suavizada: Monto",xlab="Monto",ylab="",col.main="darkblue",col.lab="darkblue",lwd=2)

hist(Monto,col="darkorange",border="white",main="Histograma: Monto",col.main="red",col.lab="blue",breaks=50,prob=T)
lines(density(Monto),col="darkblue",lwd=2)

boxplot(Monto,col="darkviolet",main="Boxplot: Monto",col.main="darkgreen",lwd=2)

Forma<- function(x) {
    n <- length(x)
    mean<-sum(x)/n
    M <-rep(0,4)
    for(i in 1:4){
    M[i] <- sum((x-mean)^i)/n
    }
    skew<-M[3]/(M[2])^(3/2)
    kurt<-(M[4]/(M[2]^2))-3
    return(list(Momentos=M,sesgo=skew,curtosis=kurt))
}

Forma(Monto)

Monto.fd<-ecdf(Monto)

plot(Monto.fd,col="magenta",main="Función de distribución empírica: Monto",xlab="x",ylab=expression(hat(F)[n](x)),col.main="blue",col.lab="darkgreen")

par(mfrow=c(2,2))

hist(Monto,col="darkorange",border="white",main="Histograma: Monto",col.main="red",col.lab="blue",breaks=50)
legend("topright", legend=c(paste("Sesgo=",round(Forma(Monto)$sesgo,4)),paste("Curtosis=",round(Forma(Monto)$curtosis,4))))


plot(density(Monto),col="darkred",main="Densidad suavizada: Monto",xlab="Monto",ylab="",col.main="darkblue",col.lab="darkblue",lwd=2)

boxplot(Monto,col="darkviolet",main="Boxplot: Monto",col.main="darkgreen",lwd=2)

plot(Monto.fd,col="magenta",main="Función de distribución empírica: Monto",xlab="x",ylab=expression(hat(F)[n](x)),col.main="blue",col.lab="darkgreen")


### Tiene cola derecha larga y pesada (Ajustamos una exponencial, que tiene la cola más ligera)

plot(qexp(ppoints(Monto),rate=1/mean(Monto)),sort(Monto),col="red",main="QQplot exponencial",xlab="",ylab="",col.main="darkblue")
abline(c(0,1),col="blue")

### Posibles densidades asociadas (no todas)

descdist(Monto,discrete=FALSE,boot=5000,obs.pch = 19, boot.col="darkviolet")


### Weibull, Lognormal, Gamma...tal vez...Pareto, Burr

source("C:/Users/Salvador/Desktop/Solvencia2024/autodistfit1.R")

fitData(Monto,c("lognormal","weibull","pareto","burr"),sample=1)

### La más mejor...BURR.

fit.burr<-fitdist(Monto, "burr",method="mle",start=list(shape1=1,shape2=1,scale=100))

ks.test(Monto, "pburr",shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])
ad.test(Monto,pburr,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])

### ¿Porqué no funcionó la gamma? mme

fitdist(Monto,"gamma",method="mle")
fitdist(Monto,"gamma",method="mme")
fitdist(Monto,"gamma",start=c(shape=0.5312323499,rate= 0.0001617227),method="mle")


plot(ecdf(Monto),verticals=TRUE,do.points=FALSE,col.hor="red", col.vert="bisque",main="Comparación entre las funciones de distribución: Monto",ylab="",col.main="darkorange",lwd=2)
curve(pgamma(x,shape=0.5312323499,rate= 0.0001617227),from=0,to=max(Monto),add=TRUE,col="blue",lwd=2)
k<-c ("Distribución empírica","Distribución teórica estimada (gamma)")
legend (30000,0.6,paste(k),lty=1,col=c("red","blue"))

plot(qgamma(ppoints(Monto),shape=0.5312323499,rate= 0.0001617227),pch=19,col="darkgreen",sort(Monto),main="QQplot: Monto",xlab="",ylab="",col.main="darkblue")
abline(c(0,1),col="magenta")


### Ajuste gráfico "a mano" de la burr

### QQplot

plot(qburr(ppoints(Monto),shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]]),pch=19,col="darkgreen",sort(Monto),main="QQplot: Monto",xlab="",ylab="",col.main="darkblue")
abline(c(0,1),col="magenta")

### Density

plot(density(Monto),type="l",col="red",main="Comparación de densidades: Monto",xlab="",ylab="",ylim=c(0,0.0003),lwd=2,col.main="magenta")
curve(dburr(x,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]]),from=0,to=max(Monto),add=TRUE,col="blue",lwd=2)
k<-c ("Densidad empírica","Densidad teórica estimada")
legend (30000,0.0001,paste(k),lty=1,col=c("red","blue"))

### FDE

plot(ecdf(Monto),verticals=TRUE,do.points=FALSE,col.hor="red", col.vert="bisque",main="Comparación entre las funciones de distribución: Monto",ylab="",col.main="darkorange",lwd=2)
curve(pburr(x,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]]),from=0,to=max(Monto),add=TRUE,col="blue",lwd=2)
k<-c ("Distribución empírica","Distribución teórica estimada")
legend (30000,0.6,paste(k),lty=1,col=c("red","blue"))

### PPplot

plot(ppoints(length(Monto)),sort(pburr(Monto,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])),pch=19,col="darkgreen",main="PPplot: Monto",
     xlab="Percentiles teóricos",ylab="Percentiles muestrales",col.main="darkblue")
abline(c(0,1),col="magenta")


### Automático

plot(fit.burr,col=rainbow(4))

### Momentos de esta burr

mburr(1,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])
mean(Monto)


mburr(2,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])-mburr(1,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])^2
var(Monto)

### como lo usamos???

burr.fde<-function(x){pburr(x,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])}

1-burr.fde(10000)

###Cómo calculamos una prima???


### Comparando los mejores ajustes: lognormal y burr

fit.lognormal<-fitdist(Monto,"lnorm")

cdfcomp(list(fit.burr, fit.lognormal), legendtext=c("burr","lognormal"))
denscomp(list(fit.burr, fit.lognormal), legendtext=c("burr","lognormal"))
qqcomp(list(fit.burr, fit.lognormal), legendtext=c("burr","lognormal"))
ppcomp(list(fit.burr, fit.lognormal), legendtext=c("burr","lognormal"))
gofstat(list(fit.burr, fit.lognormal))

### Definiendo la verosimilitud y maximizando
### The Burr distribution with parameters shape1 = a, shape2 = b and scale = s has density:
### f(x) = (a b (x/s)^b)/(x [1 + (x/s)^b]^(a + 1))
### for x > 0, a > 0, b > 0 and s > 0.
### The Burr distribution has the following special cases:
### A Loglogistic distribution when shape1 == 1;
### A Paralogistic distribution when shape2 == shape1;
### A Pareto distribution when shape2 == 1.

mlestR<-function(x,theta){-sum(dburr(x,shape1=theta[1],shape2=theta[2],scale=theta[3],log=T))}
optim(theta<-c(1,1,1),mlestR,x=Monto,method="L-BFGS-B",lower=0.001,upper=Inf)$par

mlest<- function(x,theta) {-sum(log(theta[1])+log(theta[2])+theta[2]*log(x)-theta[2]*log(theta[3])-log(x)-(theta[1]+1)*log(1+(x/theta[3])^(theta[2])))}
optim(theta<-c(1,1,1),mlest,x=Monto,method="L-BFGS-B",lower=0.001,upper=Inf)$par

### Se pueden plantear las ecuaciones de verosimilitud y resolverlas numericamente (dlogL/dtheta=0)
### En este caso, son "medio feas".


#############################################################################################################################################################
##################################################################################################################################################################
### Algunas cantidades de interés, su simulación y bootstrap

mburr(1,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])

mburr(2,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])-mburr(1,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])^2
var(Monto)

qburr(p=c(0.9,0.95,0.99), shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]],lower.tail = TRUE, log.p = FALSE)

quantile(Monto, prob=c(0.9,0.95,0.99))

### Variabilidad de estas cantidades

n<-5000
m<-5000
y<-matrix(0,n,6)
x<-matrix(0,n,m)

for(i in 1:n){
x[i,]<-rburr(m,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])
y[i,]<-c(mean(x[i,]),median(x[i,]),var(x[i,]),quantile(x[i,],probs=c(0.9,0.95,0.99)))
}

sum<-function(x){c(mean(x))}

sim1<-apply(y,2,sum)

q<-function(x){quantile(x,probs=c(0.025,0.975))}

sim2<-apply(y,2,q)


###bootstrap

library(boot)

summary.fun<-function(datos,i){
c(mean(datos[i]),median(datos[i]),var(datos[i]),quantile(datos[i],probs=0.9),quantile(datos[i],probs=0.95),quantile(datos[i],probs=0.99))
}

sum.boot<-boot(Monto,summary.fun,R=5000)

apply(sum.boot$t,2,mean)

mean.boot<-apply(sum.boot$t,2,mean)
mean.boot

q.boot<-apply(sum.boot$t,2,q)
q.boot

I.mean<-boot.ci(sum.boot,index=1)$perc[4:5]
I.median<-boot.ci(sum.boot,index=2)$perc[4:5]
I.var<-boot.ci(sum.boot,index=3)$perc[4:5]
I.q90<-boot.ci(sum.boot,index=4)$perc[4:5]
I.q95<-boot.ci(sum.boot,index=5)$perc[4:5]
I.q99<-boot.ci(sum.boot,index=6)$perc[4:5]

I.boot<-matrix(c(I.mean,I.median,I.var,I.q90,I.q95,I.q99),2,6,byrow=FALSE)
I.boot

### Tabla de comparación de resultados e intervalos de confianza

mean.mod<-mburr(1,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])
mean.mod

median.mod<-qburr(p=0.5, shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]],lower.tail = TRUE, log.p = FALSE)
median.mod

var.mod<-mburr(2,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])-mburr(1,shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]])^2
var.mod

q.mod<-qburr(p=c(0.9,0.95,0.99), shape1=fit.burr$estimate[[1]], shape2=fit.burr$estimate[[2]],scale=fit.burr$estimate[[3]],lower.tail = TRUE, log.p = FALSE)
q.mod

mean.dat<-mean(Monto)
mean.dat

median.dat<-median(Monto)

var.dat<-var(Monto)
var.dat

q.dat<-quantile(Monto, prob=c(0.9,0.95,0.99))
q.dat

options(scipen=999)
Resultados<-matrix(c(mean.dat,median.dat,var.dat,q.dat,mean.mod,median.mod,var.mod,q.mod,sim1,mean.boot,sim2[1,],sim2[2,],I.boot[1,],I.boot[2,]),6,8)
colnames(Resultados)<-c("Datos","Modelo","Simulación","Bootstrap","simLi","simLs","bootLi","bootLs")
rownames(Resultados)<-c("Media","Mediana","Varianza","q90","q95","q99")
Resultados

###########################################################################################################################################################
################################################### VALORES EXTREMOS ##############################################################################
### VALORES EXTREMOS
### Ajuste del modelo GPD a los datos de la SOA

montos<-read.table("C:/Users/Salvador/Desktop/Cursos2022-I/SolvenciaII-2021/SOA91.TXT",sep=",")

summary(montos[,4])

par(mfrow=c(1,2))

plot(density(montos[,4]),col="darkblue",xlab="",ylab="Densidad suavizada",col.lab="darkred",main="Densidad tipo kernel",col.main="darkgreen")

plot(density(log(montos[,4])),col="darkblue",xlab="",ylab="Densidad suavizada",col.lab="darkred",main="Densidad tipo kernel(log)",col.main="darkgreen")


par(mfrow=c(1,2))

hist(montos[,4],col="darkblue",border="white",main="Histograma montos de reclamación")

truehist(log(montos[,4]),col="darkgreen",border="white",main="Histograma montos de log(reclamación)")


par(mfrow=c(1,2))

boxplot(montos[,4],col="red",main="Montos")

boxplot(log(montos[,4]),col="blue",main="Log(montos)")


skew(montos[,4]);kurtosi(montos[,4])

skew(log(montos[,4]));kurtosi(log(montos[,4]))

### Grafica de la funcion media de excesos

mrlplot(montos[,4],col="darkgreen",conf=FALSE)

abline(v=230000,lty=2,col="blue")

abline(v=215000,lty=3,col="red")

abline(v=200000,lty=4,col="green")


mrlplot(montos[,4],lty=rep(1,3),col=c("darkblue","darkgreen","darkblue"))
abline(v=230000,lty=2,col="blue")

abline(v=215000,lty=3,col="red")

abline(v=200000,lty=4,col="green")

###Extra

mrlplot(montos[,4],col="darkgreen",conf=FALSE,xlim=c(0,750000))

abline(v=230000,lty=2,col="blue")

abline(v=215000,lty=3,col="red")

abline(v=200000,lty=4,col="green")

abline(v=208000,lty=4,col="violet")


### Será más clara con el logaritmo?

lmontos<-log(montos[,4])

mrlplot(lmontos,lty=rep(1,3),col=c("darkblue","darkgreen","darkblue"))

### Confirmando con otra gráfica

mrl.plot(montos[,4], umin = min(montos[,4]), umax = max(montos[,4]) - 0.1,conf = 0.95, nint = 100)
abline(v=200000,lty=3,col="red")


### Gráfica de estabilidad de los parámeros
### Gráfica de estabilidad de los parámeros

source("C:/Users/Salvador/Desktop/Cursos2022-I/SolvenciaII-2021/parStabilityPlot.R")
source("C:/Users/Salvador/Desktop/Cursos2022-I/SolvenciaII-2021/pareto2.goftest.R")

parStabilityPlot(montos[,4], c(1000,500000))


### Ajuste de un modelo GPD con un umbral de 210000

### Libreria evir

u<-210000

fit<-gpd(montos[,4],u)

plot.gpd(GPD.fit)

ks.test(montos[,4],"pgpd",shape=fit$par.ests[[1]],scale=fit$par.ests[[2]])

ks.test(montos[,4][montos[,4]>u],"pgpd",shape=fit$par.ests[[1]],scale=fit$par.ests[[2]])

pareto2.goftest(montos[,4], fit$par.ests[[2]],fit$par.ests[[1]], method="anderson-darling")

pareto2.goftest(montos[,4], fit$par.ests[[2]],fit$par.ests[[1]], method="kolmogorov")

pareto2.goftest(montos[,4][montos[,4]>u], fit$par.ests[[2]],fit$par.ests[[1]], method="anderson-darling")

pareto2.goftest(montos[,4][montos[,4]>u], fit$par.ests[[2]],fit$par.ests[[1]], method="kolmogorov")

###Sera que el tamanio de muestra tan grande esta influyendo en estas pruebas de bondad de ajuste?

samp.dat<-sample(montos[,4],10000)

fit1<-gpd(samp.dat,u)

plot.gpd(fit1)

ks.test(samp.dat,"pgpd",shape=fit1$par.ests[[1]],scale=fit1$par.ests[[2]])

ks.test(samp.dat[samp.dat>u],"pgpd",shape=fit1$par.ests[[1]],scale=fit1$par.ests[[2]])

pareto2.goftest(samp.dat, fit1$par.ests[[2]],fit1$par.ests[[1]], method="anderson-darling")

pareto2.goftest(samp.dat, fit1$par.ests[[2]],fit1$par.ests[[1]], method="kolmogorov")

##!!!Que #$##&%"@##$% esta pasando !!!?

###Está pasando algo extraño. ¿Porqué si se ve tan bien ajustada, las estadísticas no reflejan este hecho?
###Porque estas estadísticas comparan SIEMPRE contra la distribución completa, que inicia para valores >=0, no en los umbrales

###Evaluacion a traves de simulacion

gpdAd(montos[,4])
gpdCvm(montos[,4])

gpdAd(montos[,4][montos[,4]>u])

gpdCvm(montos[,4][montos[,4]>u])

###Con un paquete hecho para esta tarea (Con distintos umbrales)

est1<-gpd.fit(montos[,4],threshold =210000)$mle

gnfit(montos[,4], "gpd", pr = est1,threshold =210000)


est2<-gpd.fit(montos[,4],threshold =200000)$mle

gnfit(montos[,4], "gpd", pr = est2,threshold =200000)


est3<-gpd.fit(montos[,4],threshold =215000)$mle

gnfit(montos[,4], "gpd", pr = est3,threshold =215000)


est4<-gpd.fit(montos[,4],threshold =230000)$mle

gnfit(montos[,4], "gpd", pr = est4,threshold =230000)

###Como observamos, el "mas mejor" ajuste se logra con un umbral de 210000

###Simulamos una Pareto Generalizada con los parametros estimados
###Calculamos medidas de variabilidad para VaR y TVaR

###Hagamoslo

###Primero VaR y TVaR exactos

shape<-fit$par.ests[[1]]; scale<-fit$par.ests[[2]] ; u<-fit$threshold; n<-fit$n; nu<-fit$n.exceed; alpha<-0.95

c(shape,scale,u,n,nu,alpha)

VaR.GPD<-u+(scale/shape)*((n/nu*(1-alpha))^(-shape)-1)
VaR.GPD

TVaR.GPD<-VaR.GPD/(1-shape)+(scale-shape*u)/(1-shape)
TVaR.GPD


###Empiricos

quantile(montos[,4],0.95);mean(montos[,4][montos[,4]>VaR.GPD])

###Simulando (que es gerundio). Metodo de inversion

n<-5000
m<-10000
y<-matrix(0,n,2)
x<-matrix(0,n,m)
umbral<-fit$threshold
nn<-fit$n

for(i in 1:n){
k<-floor(0.95*m)+1
x[i,]<-sort(umbral+scale/shape*((nn/nu*runif(m))^(-shape)-1))
y[i,]<-c(quantile(x[i,],0.95),mean(x[i,][k:m]))
}

simGPD<-apply(y,2,mean)

q<-function(x){quantile(x,probs=c(0.025,0.975))}

sim2<-apply(y,2,q)

simGPD

sim2

#####
#########################################################################################################################################################
############################################################# VaR y TVaR ################################################################################

### Comparacion del VaR entre una distribucion con colas ligeras y una con colas pesadas

curve(dgamma(x,shape=5,scale=1/3),from=0,to=10,col="darkred",xlab="x",ylab="",
main="Comparacion colas distribuciones Gamma y Pareto",col.main="darkgreen",col.lab="darkorange")

curve(dpareto(x,shape=6/5,scale=1/3),from=0,to=10,col="darkblue",add=T)

k<-c ("Gamma (5,1/3)","Pareto (6/5,1/3)")
legend (5,0.5,paste(k),lty=1, col=c("darkred","darkblue"))


qgamma(c(0.95,0.99,0.995),shape=5,scale=1/3)

qpareto(c(0.95,0.99,0.995),shape=6/5,scale=1/3)

###Comparacion de funciones de supervivencia

curve(1-pgamma(x,shape=5,scale=1/3),from=0,to=10,col="red",main="Comparacion de supervivencias",col.main="darkviolet",ylab="")

curve(1-ppareto(x,shape=6/5,scale=1/3),from=0,to=10,col="blue",add=T)

k<-c ("Sup Gamma (5,1/3)","Sup Pareto (6/5,1/3)")
legend (5,0.5,paste (k),lty=1, col=c("red","blue"))


#### Comparacion de funciones de riesgo

curve(dgamma(x,shape=5,scale=1/3)/(1-pgamma(x,shape=5,scale=1/3)),from=0,to=10,col="red",ylab="",
      main="Comparaci�n de funciones de riesgo",col.main="darkorange")

curve(dpareto(x,shape=6/5,scale=1/3)/(1-ppareto(x,shape=6/5,scale=1/3)),from=0,to=10,col="blue",ylab="",add=T)

k<-c ("Riesgo Gamma (5,1/3)","Riesgo Pareto (6/5,1/3)")
legend (5,0.8,paste (k),lty=1, col=c("red","blue"))

### Simulacion para dar una medida de variabilidad al VaR de estas distribuciones
### Gamma
n<-1000
m<-1000
y<-rep(0,n)
x<-matrix(0,n,m)

for(i in 1:n){
x[i,]<-rgamma(m,shape=5,scale=1/3)
y[i]<-quantile(x[i,],probs=0.95)

}

c(mean(y),quantile(y,probs=c(0.025,0.975)))

### Pareto

y1<-rep(0,n)
x1<-matrix(0,n,m)

for(i in 1:n){
x1[i,]<-rpareto(m,shape=6/5,scale=1/3)
y1[i]<-quantile(x1[i,],probs=0.95)

}

c(mean(y1),quantile(y1,probs=c(0.025,0.975)))


### Simulacion para dar una medida de variabilidad al TVaR de estas distribuciones

#### TVaR(0.95) Gamma

n<-1000
m<-1000
k<-rep(0,n)
y2<-rep(0,n)
x2<-matrix(0,n,m)

for(i in 1:n){

x2[i,]<-sort(rgamma(m,shape=5,scale=1/3))
k<-floor(0.95*m)+1
y2[i]<-sum(x2[i,][k:m])/(m-k+1)

}

c(mean(y2),quantile(y2,probs=c(0.025,0.975)))

### TVaR(0.95) Pareto

y3<-rep(0,n)
x3<-matrix(0,n,m)

for(i in 1:n){

x3[i,]<-sort(rpareto(m,shape=6/5,scale=1/3))
k<-floor(0.95*m)+1
y3[i]<-mean(x3[i,][x3[i,]>=3.705253])

}

c(mean(y3),quantile(y3,probs=c(0.025,0.975)))

###VaR y TVaR pareto analiticos

(1/3)*((1-0.95)^(-5/6)-1)

(1/3)*((1-0.95)^(-5/6)-1)+((1/3)*(1-0.95)^(-5/6))/(6/5-1)

###############################################################################################################################################################
###############################################################################################################################################################
### Graficas con modificaciones de covertura

### Pago del pago: Y^P

f<-coverage(dpareto,ppareto,deductible=500)
curve(f(x,shape=3,scale=2000),xlab="x",ylab="f(x)",xlim=c(0.01,5000),main="Pareto(3,2000)",col="red",col.main="darkred")
curve(dpareto(x, shape=3, scale=2000), col="blue", add=TRUE)
t<- c("X",expression(Y^P))
legend(3000, 0.0008, paste(t), lty=1, col=c("blue", "red"))

### Pago de la perdida: Y^L

g<-coverage(dpareto,ppareto,deductible=500,per.loss=TRUE)
curve(g(x, shape=3, scale=2000), xlab="x", ylab="f(x)",xlim=c(0.0001,5000),main="Pareto(3,2000)",col="red",col.main="darkred")
curve(dpareto(x, shape=3, scale=2000), col="blue", add=TRUE)
points(0, g(0,3,2000), pch=19, col="red")
t<- c("X", expression(Y^L))
legend(3000, 0.0004, paste(t), lty=1, col=c("blue", "red"))


### Severidad

### Pareto 3, 2000, d=500

### Vimos que la nueva variable es una Pareto(3,2500)

curve(dpareto(x,shape=3,scale=2000),from=0,to=5000,col="darkred",xlab="x",ylab="",
      main="Comparacion de las distribuciones Pareto sin y con deducible",col.main="darkgreen",col.lab="darkorange")

curve(dpareto(x,shape=3,scale=2500),from=0,to=5000,col="darkblue",add=T)

k<-c ("Sin deducible: Pareto(3,2000)","Con deducible: Pareto(3,2500)")
legend (2800,0.0008,paste(k),lty=1, col=c("darkred","darkblue"),cex=1.3)



### Funciones de los modelos modificados por la presencia de un deducible

### Variable de pago

FYP<-coverage(cdf=ppareto,deductible=500,limit=Inf)

FYP(500,3,2000)

curve(FYP(x,3,2000),0,500)

### Sin utilizar las funciones de actuar. Esta funcion corresponde a

Fyp1<-function(y){1-(2500/(2500+y))^(3)}

Fyp1(500)

curve(Fyp1(x),0,500)


### variable de perdida

FYL<-coverage(cdf=ppareto,deductible=500,limit=Inf,per.loss=TRUE)

FYL(0,shape=3,scale=2000)


curve(FYP(x,shape=3,scale=2000),from=0,to=4000,col="darkred")
curve(FYL(x,shape=3,scale=2000),from=0,to=4000,col="darkblue", add=T)
k<-c ("FYP","FYL")
legend (2500,0.4,paste (k),lty=1, col=c("darkred","darkblue"))

### Densidades

fYP<-coverage(dpareto,cdf=ppareto,deductible=500, limit=Inf)
fYL<-coverage(dpareto,cdf=ppareto,deductible=500,limit=Inf,per.loss=TRUE)

curve(fYP(x,shape=3,scale=2000),from=0,to=4000,col="darkgreen",ylab="")
curve(fYL(x,shape=3,scale=2000),from=0,to=4000,col="darkviolet", add=T,ylab="")
k<-c ("fYP","fYL")
legend (2000,0.0006,paste (k),lty=1, col=c("darkgreen","darkviolet"))

### Momentos restringidos y momentos sin restriccion

mpareto(shape=3,scale=2000,order=1)

levpareto(500,shape=3,scale=2000,order=1)

### Funcion con todas las modificaciones consideradas.

fyLI<-coverage(dpareto,ppareto,limit=3000,inflation=0.1,deductible=500,coinsurance=0.8,per.loss = TRUE)

curve(fyLI(x,3,2000),from=500,to=1500)

fyLI(500,3,2000)

fyLI(500,3,2000);fyLI(3000,3,2000)























